### Download Flowers dataset

In [1]:
#You can download the data manually as well instead of using 'wget'
!wget http://download.tensorflow.org/example_images/flower_photos.tgz --quiet

In [2]:
#Check if file is downloaded
!ls -l

total 223456
-rw-r--r-- 1 root root 228813984 Feb 10  2016 flower_photos.tgz
drwxr-xr-x 1 root root      4096 Jun 20 18:46 sample_data


In [3]:
#Unzip the data
!tar -xf flower_photos.tgz

In [4]:
#Check how data is organized
!ls -l flower_photos

total 616
drwx------ 2 270850 5000  36864 Feb 10  2016 daisy
drwx------ 2 270850 5000  45056 Feb 10  2016 dandelion
-rw-r----- 1 270850 5000 418049 Feb  9  2016 LICENSE.txt
drwx------ 2 270850 5000  36864 Feb 10  2016 roses
drwx------ 2 270850 5000  36864 Feb 10  2016 sunflowers
drwx------ 2 270850 5000  40960 Feb 10  2016 tulips


### Build batch generator

In [5]:
import tensorflow as tf

In [6]:
#Define some parameters
img_size = 60
img_depth = 3

Create an ImageDataGenerator object, it can also split data between train and test.

In [7]:
#ImageDataGenerator declaration with 20% data as test (80% for training)
img_generator= tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

ImageDataGenerator can read images directory and also resize them if needed

In [8]:
#Build training generator.
train_generator = img_generator.flow_from_directory('flower_photos',
                                                    batch_size=64,
                                                    target_size=(img_size, img_size),
                                                    subset='training')

Found 2939 images belonging to 5 classes.


In [9]:
#Build test generator
test_generator = img_generator.flow_from_directory('flower_photos',
                                                   target_size=(img_size, img_size),
                                                   subset='validation',
                                                   batch_size=64)

Found 731 images belonging to 5 classes.


In [10]:
type(train_generator)

keras.src.preprocessing.image.DirectoryIterator

ImageDataGenerator returns 64 images and their labels

In [11]:
#Lets check the features (images) and Labels (flower class) returned by ImageDataGenerator
X, y = next(train_generator)

In [12]:
print('Input features shape', X.shape)
print('Actual labels shape', y.shape)

Input features shape (64, 60, 60, 3)
Actual labels shape (64, 5)


In [13]:
y[0]

array([0., 0., 0., 0., 1.], dtype=float32)

In [14]:
import numpy as np
np.unique(X[0])

array([  8.,  16.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,
        28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,
        39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,
        50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,
        61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,
        72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,
        83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,
        94.,  95.,  96.,  97.,  98.,  99., 100., 101., 102., 103., 104.,
       105., 106., 107., 108., 109., 110., 111., 112., 113., 114., 115.,
       116., 117., 118., 119., 120., 121., 122., 123., 124., 125., 126.,
       127., 128., 129., 130., 131., 132., 133., 134., 135., 136., 137.,
       138., 139., 140., 141., 142., 143., 144., 145., 146., 147., 148.,
       149., 150., 151., 152., 153., 154., 155., 156., 157., 158., 159.,
       160., 161., 162., 163., 164., 165., 166., 16

In [15]:
train_generator.class_indices

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [ ]:
X[0]

### Build CNN Model

In [16]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(32,
                                 kernel_size=(3,3),
                                 activation='relu'))

In [17]:
model.output

<KerasTensor: shape=(None, 58, 58, 32) dtype=float32 (created by layer 'conv2d')>

In [18]:
#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.25))

#Add Output Layer
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [19]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
#Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 60, 60, 3)         12        
 Normalization)                                                  
                                                                 
 conv2d (Conv2D)             (None, 58, 58, 32)        896       
                                                                 
 batch_normalization_1 (Bat  (None, 58, 58, 32)        128       
 chNormalization)                                                
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 batch_normalization_2 (Bat  (None, 56, 56, 64)        256       
 chNormalization)                                                
                                                        

### Train the model

In [21]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('flowers.h5',
                                                      save_best_only=True,
                                                      monitor='val_accuracy',
                                                      mode='max',
                                                      verbose=1)

In [22]:
2939//64

45

In [24]:
model.fit(train_generator,
          epochs=20,
          steps_per_epoch= 2939//64,  #Number of batches per epoch
          validation_data=test_generator,
          validation_steps = 731//64,
          callbacks=[model_checkpoint]) #Number of test images//batch_size

Epoch 1/20
45/45 [==============================] - ETA: 0s - loss: 1.4363 - accuracy: 0.3920
Epoch 1: val_accuracy improved from -inf to 0.26136, saving model to flowers.h5
45/45 [==============================] - 66s 1s/step - loss: 1.4363 - accuracy: 0.3920 - val_loss: 12.2734 - val_accuracy: 0.2614


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/20
45/45 [==============================] - ETA: 0s - loss: 1.2762 - accuracy: 0.4946
Epoch 2: val_accuracy did not improve from 0.26136
45/45 [==============================] - 65s 1s/step - loss: 1.2762 - accuracy: 0.4946 - val_loss: 29.2677 - val_accuracy: 0.2216
Epoch 3/20
45/45 [==============================] - ETA: 0s - loss: 1.1613 - accuracy: 0.5134
Epoch 3: val_accuracy did not improve from 0.26136
45/45 [==============================] - 63s 1s/step - loss: 1.1613 - accuracy: 0.5134 - val_loss: 33.7433 - val_accuracy: 0.2472
Epoch 4/20
45/45 [==============================] - ETA: 0s - loss: 1.0453 - accuracy: 0.5607
Epoch 4: val_accuracy improved from 0.26136 to 0.28835, saving model to flowers.h5
45/45 [==============================] - 63s 1s/step - loss: 1.0453 - accuracy: 0.5607 - val_loss: 21.9670 - val_accuracy: 0.2884
Epoch 5/20
45/45 [==============================] - ETA: 0s - loss: 1.0215 - accuracy: 0.5920
Epoch 5: val_accuracy improved from 0.28835 to 0.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 100, 100, 3)      12        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 batch_normalization_1 (Batc  (None, 98, 98, 32)       128       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 98, 98, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 64)        18496     
                                                                 
 batch_normalization_2 (Batc  (None, 96, 96, 64)       2

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 60, 60, 3)        12        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 58, 58, 32)        896       
                                                                 
 batch_normalization_1 (Batc  (None, 58, 58, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 batch_normalization_2 (Batc  (None, 56, 56, 64)       256       
 hNormalization)                                                 
                                                        